# Mortality Group Weekly Report - 10/14/2020
Rustom Ichhaporia

I have included most of the text from my midterm report here. 

## Overview

The mortality section of the project aims to predict the mortality of a person given their current socioeconomic factors such as health, income, and family status. Although no long-term mortality prediction can be perfectly accurate, getting a probabilistic sense of life expectancy can be useful for financial planning. For example, it allows someone to optimize their portfolio with a more accurate expectation of when they will enter retirement. 

At the beginning, I was assigned to work in a group, but my partner dropped out of the research program so I have been working by myself. 

## Preliminary Exploration

For the first few weeks, I attempted to work with a dataset from the CDC recommended by Linfeng Zhang. The dataset was called Mortality Multiple Cause-of-Death Public Use Record and is linked below. It contained location, family, date, and cause of death features for almost every death in the United States for the past few decades. I began by working with reading in and transforming the data, which was time consuming because of the complex nature of the data guide (it was not written in a standard table). Some of this work was documented in earlier weekly reports. However, this dataset did not contain information about the income or occupation of the deceased, which is an important element of our analysis. We considered simply combining predictions from an income model and a health model, but decided against this because it is statistically dubious and may have ignored interaction terms that would affect results. The code from this portion of my work has been omitted from this report, but some of it can be found in my earlier weekly reports. 

https://www.cdc.gov/nchs/nvss/mortality_public_use_data.htm

## New Dataset Search 

After deciding to move on from the CDC dataset, I began looking into datasets that combined all three of our relevant items: economic status, health status, and the target variable, mortality. This was somewhat difficult, because most publicly available, large-scale datasets in this area only contain two of those three variables. At one weekly meeting, we discussed the potential of combining datasets as described above using a statistical methods such as hierarchical modeling and multiple frames as described in the paper linked below which was recommended by Yong Xie. However, I decided to continue looking for datasets and found the dataset from the National Longitudinal Mortality Study that incorporates all three of the variables mentioned above. After applying for approval and getting access to it, I began to work with the data. While it has more limited health information and fewer data points, I will attempt to extract meaningful insights from the data. 

https://projecteuclid.org/euclid.ss/1494489817

## National Longitudinal Mortality Study (NLMS) Dataset

The most recent iteraction of the NLMS dataset linked below contains roughly 3.8 million records with 550 thousand 1.8 million mortality entries, but my sample of the data contains roughly 1.8 million entries, and 100 thousand mortality entries. The dataset follows the format of a follow-up study, in which data about the participants was collected in 1990 and their potential mortality was observed 11 years later. There are more datasets with more recent data, but they are smaller, and the process for incorporating the more recent data should not be difficult once the old data has been properly modeled. 

The data contains roughly 43 features, including age, race, sex, zip code, marital status, number of members of the household, highest education, health rating, employment status/occupation, income, smoking status, and cause of death (if applicable). I have used a selection of these variables which seem most relevant, leaving out items like health insurance type and day of week of death. These can be added later if deemed useful. 

https://www.census.gov/topics/research/nlms.Reference_Manual.html

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import tree
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import average_precision_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.utils import resample
from sklearn.metrics import auc
sns.set()

In [2]:
df_raw = pd.read_csv('data/11.csv').drop(['smok100', 'agesmk', 'smokstat', 'smokhome', 'curruse', 'everuse'], axis=1)
df_raw

,record,age,race,sex,ms,hisp,adjinc,educ,pob,wt,...,vt,histatus,hitype,povpct,stater,rcow,tenure,citizen,health,indalg
0,88426,70,1.0,2,5.0,3.0,11.0,4.0,909,151,...,0.0,NaN,NaN,18,16,4.0,1.0,NaN,NaN,1.0
1,88427,79,1.0,2,2.0,3.0,11.0,4.0,909,132,...,0.0,NaN,NaN,18,16,3.0,1.0,NaN,NaN,NaN
2,88428,34,1.0,1,1.0,3.0,8.0,4.0,909,155,...,0.0,NaN,NaN,10,16,1.0,2.0,NaN,NaN,1.0
3,88429,32,1.0,2,1.0,3.0,8.0,1.0,909,155,...,0.0,NaN,NaN,10,16,1.0,2.0,NaN,NaN,1.0
4,88430,2,1.0,2,NaN,3.0,8.0,NaN,909,145,...,NaN,NaN,NaN,10,16,NaN,2.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1835067,666,19,1.0,1,5.0,2.0,4.0,8.0,909,60,...,0.0,1.0,4.0,6,16,2.0,2.0,NaN,1.0,NaN
1835068,667,33,1.0,2,1.0,2.0,11.0,6.0,909,56,...,0.0,1.0,4.0,10,16,NaN,2.0,1.0,1.0,NaN
1835069,668,16,1.0,2,5.0,2.0,11.0,6.0,909,60,...,0.0,1.0,4.0,10,16,NaN,2.0,NaN,1.0,NaN
1835070,669,7,1.0,2,NaN,2.0,11.0,NaN,909,51,...,NaN,1.0,4.0,10,16,NaN,2.0,1.0,1.0,NaN


Because this data takes a long time to compile and verify from multiple smaller studies, the creators included both a definite and an algorithmic indicator of death to speed up the release of the data. These features, `'inddea'` and `'indalg'`, respectively, have been combined through intersection into one target variable, `'indmort'` in the way that the reference manual recommends. The expanded list of features is below. 

In [3]:
numerical = ['age', 'hhnum']
uneven_numerical = ['adjinc', 'health', 'follow']
categorical = ['race', 'sex', 'ms', 'hisp', 'educ', 'pob', 'hhid', 'reltrf', 'occ', 'majocc', 'ind', 'esr', 'urban', 'smsast', 'inddea', 'cause113', 'dayod', 'hosp', 'hospd', 'ssnyn', 'vt', 'histatus', 'hitype', 'povpct', 'stater', 'rcow', 'tenure', 'citizen', 'indalg']
smoking = ['smok100', 'agesmk', 'smokstat', 'smokhome', 'curruse', 'everuse']
misc = ['record', 'wt']


<pre>List of all the features and their full names is pasted below. For the full description of the features, refer to the read_pubfile5.dat file. 
@1 record     $   7.     /*Record Number (page no. 6)                              */
    @8 age            2.     /*Age at Time of Interview (page no. 10)                  */
    @10 race      $   1.     /*Race  (page no.12)                                      */
    @11 sex       $   1.     /*Sex   (page no.10)                                      */
    @12 ms        $   1.     /*Marital Status (page no.13)                             */
    @13 hisp      $   1.     /*Hispanic Origin (page no. 12)                           */
    @14 adjinc    $   2.     /*Inflation Adjusted Income (page no.20)                  */
    @16 educ      $   2.     /*Highest Grade Completed (page no.14)                    */
    @18 pob       $   3.     /*Region of Birth (page no. 11)                           */
    @21 wt            4.     /*Adjusted Weight (page no. 6 )                           */
    @25 hhid      $   7.     /*Household ID No. (page no. 6)                           */
    @32 hhnum         2.     /*Number of People in HH (page no. 14)                    */
    @34 reltrf    $   1.     /*Relationship to Reference Person (page no.13)           */
    @35 occ       $   4.     /*4 Digit Occupation Code (page no. 18)                   */
    @39 majocc    $   2.     /*Major Occupation Code (page no. 18 )                    */
    @41 ind       $   4.     /*4 Digit Industry Code (page no. 17)                     */
    @45 majind    $   2.     /*Major Industry Code (page no. 18)                       */
    @47 esr       $   1.     /*Employment Status Recode (page no. 17)                  */
    @48 urban     $   1.     /*Urban/Rural Status (page no. 8)                         */
    @49 smsast    $   1.     /*SMSAST Status (page no.9)                               */
    @50 inddea    $   1.     /*Death Indicator (page no. 23)                           */
    @51 cause113  $   3.     /*Cause of Death (page no. 23)                            */
    @54 follow        4.     /*Length of Follow-up (page no. 24)                       */
    @58 dayod     $   1.     /*Day of Week of Death (page no. 24)                      */
    @59 hosp      $   1.     /*Hospital Type (page no.25)                              */
    @60 hospd     $   1.     /*Hospital Death Indicator (page no. 26)                  */
    @61 ssnyn     $   1.     /*Presence of SSN (page no. 7)                            */
    @62 vt        $   1.     /*Veteran Status (page no. 16)                            */
    @63 histatus  $   1.     /*Health Insurance Status (page no. 22)                   */
    @64 hitype    $   1.     /*Health Insurance Type (page no. 22)                     */
    @65 povpct    $   2.     /*Income as Percent of Poverty Level (page no. 21)        */
    @67 stater    $   2.     /*State Recode (page no. 8)                               */
    @69 rcow      $   2.     /*Recoded Class of Worker (page no.19)                    */
    @71 tenure    $   1.     /*Housing Tenure (page no. 14)                            */
    @72 citizen   $   1.     /*Citizenship (page no. 16)                               */
    @73 health    $   2.     /*Health (page no. 16)                                    */
    @75 indalg        1.     /*Indicator of Algorithmic Death (page no. 27)            */
    @76 smok100   $   1.     /*Smoked More than 100 Cigarettes (page no. 28)           */
    @77 agesmk    $   2.     /*Age Started Smoking (page no. 28)                       */
    @79 smokstat  $   1.     /*Cigarette Smoking Status (page no.28)                   */
    @80 smokhome  $   1.     /*Rules for Smoking Cigarettes in the Home (page no. 29 ) */
    @81 curruse   $   5.     /*Currently Use Smokeless Tobacco (page no. 30)           */
    @86 everuse   $   5.     /*Ever Use Smokeless Tobacco (page no. 31)                */</pre>

In [4]:
# indmort is the recommended combination feature of both confirmed deaths and computer-predicted deaths based on the data collection agency
df_raw['indmort'] = df_raw['inddea'][(df_raw['inddea'] == 1) & (df_raw['indalg'] == 1)]
df_raw['indmort'] = df_raw['indmort'].fillna(0)
df_raw['indmort'].sum()

94107.0

Each entry also has a weight that is meant to highlight the fact that not every entry is representative of the same number of people. Hidden statistical smoothing has been used so that the applying the weights to each entry will yield a more accurate estimate of societal data, but I have not yet made use of the weights. 

In [5]:
# "Weight" of entry, roughly 50-200. I am not sure how to apply these to the data. 
df_raw.wt.describe()

count    1.835072e+06
mean     1.328667e+02
std      7.247297e+01
min      0.000000e+00
25%      7.600000e+01
50%      1.340000e+02
75%      1.790000e+02
max      1.522000e+03
Name: wt, dtype: float64

In [6]:
# Selection of fewer variables for EDA purposes
# Remove cause113 because it is not predictive
used_features = ['age', 'hhnum', 'adjinc', 'health', 'occ', 'ind', 'esr', 'ms', 'indmort']
df = df_raw[used_features]

In [7]:
df.isna().sum() / df.shape[0]

age        0.000000
hhnum      0.000000
adjinc     0.024124
health     0.790674
occ        0.466099
ind        0.466219
esr        0.191220
ms         0.196846
indmort    0.000000
dtype: float64

In [8]:
mort_corr = df.corr()['indmort'].sort_values()
mort_corr

hhnum     -0.169388
adjinc    -0.098752
ms        -0.073020
ind       -0.010118
occ        0.004227
esr        0.195555
health     0.282516
age        0.336753
indmort    1.000000
Name: indmort, dtype: float64

Correlations can be useful for numerical features, but most of these features are categorical, so I decided to begin one-hot encoding and imputation of missing values. Most of the health rating entries are still missing. 

In [9]:
df = df.astype({'occ':'category', 'ind': 'category', 'esr': 'category', 'ms': 'category'})
df.dtypes

age           int64
hhnum         int64
adjinc      float64
health      float64
occ        category
ind        category
esr        category
ms         category
indmort     float64
dtype: object

In [10]:
# df = pd.get_dummies(df, dummy_na=True)

Recently, I created a decision tree classifier to attempt to model the data but my results were likely very unhelpful considering the number of other items I have yet to consider to polish the data before training a model on it. I used a train-test-split and calculated the mean squared error, but I will use more formal cross-fold validation later. I had previously been filling the NaN values in the Dataframe with the means of their respective features, but I was advised that this would provide a poor estimation, especially considering the large number of missing entries. To fix this, I will begin looking through the weekly reports of other groups to see what methods they used to resolve these issues and try to implement some of them myself. 

It seems like the method of imputation that other groups were using was most commonly filling in the NaN values with a specific NaN substitute, but I'm not sure how to apply this to my numerical values or what the threshold should be for dropping entries, since many of them do not contain entries for the health feature. 

I am still unsure whether to use a classifier or a regression model. Although the output of death is binary, the it would be more useful in the construction of a death age probability distribution to get a likelihood score of death (e.g. 0.65% chance dead at age 70) instead of a binary response (e.g. dead or not dead at age 70), and so a logistic regression would be useful. However, it was suggested that because death is a categorical variable, I should use a classifier. I would appreciate feedback on which one I should attempt to begin working with. 

I apologize for my slow progress, as I am working by myself and have not learned many of the advanced methods that some other groups are using. I have excluded the code for my decision tree and some of the other EDA that I conducted earlier for the sake of clarity, but some of it can be found in my earlier weekly reports. This is part of the reason for the brevity of my report. Some of the methods that Frank Quan recommended I use are lightgbm, auc, pr auc, and recall precision f1 score, all of which I will look into in the coming weeks. 

Ultimately, I hope to hope to have a model that, given the input parameters specified, can determine the probability distribution of a person's future date of death. Any help on what type of model would be ideal for this data or what method of imputation would be best would be appreciated. Thank you. 

Here, I begin my progress after the midterm report. I decided that using precision recall area under curve (PR AUC) would be the best method of testing the accuracy of a potential model because it is the preferred metric when the data is heavily imabalanced and when the positive class is more important than the negative class (both of these conditions apply to my dataset). This was based on this article: https://neptune.ai/blog/f1-score-accuracy-roc-auc-pr-auc. 

I still am not really sure what the best way for me to impute the values is as I have not really been able to discover this from looking at the reports of other groups. However, after doing some of my own research, I found that 

In [11]:
df

,age,hhnum,adjinc,health,occ,ind,esr,ms,indmort
0,70,2,11.0,NaN,2630.0,5470.0,1.0,5.0,0.0
1,79,2,11.0,NaN,4700.0,5470.0,1.0,2.0,0.0
2,34,3,8.0,NaN,8960.0,2980.0,1.0,1.0,0.0
3,32,3,8.0,NaN,8960.0,5470.0,1.0,1.0,0.0
4,2,3,8.0,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...
1835067,19,2,4.0,1.0,4760.0,4770.0,1.0,5.0,0.0
1835068,33,6,11.0,1.0,NaN,NaN,5.0,1.0,0.0
1835069,16,6,11.0,1.0,NaN,NaN,5.0,5.0,0.0
1835070,7,6,11.0,1.0,NaN,NaN,NaN,NaN,0.0


In [12]:
df_raw['citizen'].value_counts(dropna=False)

NaN    1370211
1.0     406059
5.0      34765
4.0      16009
3.0       4167
2.0       3861
Name: citizen, dtype: int64

In [13]:
df_raw['hosp'].value_counts(dropna=False)

NaN    1677278
1.0      91819
4.0      34968
2.0      25142
5.0       5308
6.0        387
3.0        170
Name: hosp, dtype: int64

In [14]:
(df_raw.isna().sum() / df_raw.shape[0]).sort_values()

record      0.000000
stater      0.000000
povpct      0.000000
ssnyn       0.000000
follow      0.000000
cause113    0.000000
inddea      0.000000
hhnum       0.000000
hhid        0.000000
indmort     0.000000
pob         0.000000
age         0.000000
sex         0.000000
wt          0.000000
race        0.001575
reltrf      0.002547
urban       0.007107
smsast      0.007115
tenure      0.013867
adjinc      0.024124
hisp        0.028610
educ        0.191202
esr         0.191220
ms          0.196846
vt          0.215823
histatus    0.314846
hitype      0.314846
rcow        0.465219
majocc      0.466099
occ         0.466099
majind      0.466219
ind         0.466219
citizen     0.746680
health      0.790674
indalg      0.809948
dayod       0.912401
hosp        0.914012
hospd       0.920889
dtype: float64

In [15]:
# indmort is the recommended combination feature of both confirmed deaths and computer-predicted deaths based on the data collection agency
df_raw['indmort'] = df_raw['inddea'][(df_raw['inddea'] == 1) & (df_raw['indalg'] == 1)]
df_raw['indmort'] = df_raw['indmort'].fillna(0)
df_raw['indmort'].sum()

94107.0

In [16]:
numerical = ['age', 'hhnum', 'povpct']
uneven_numerical = ['adjinc', 'health', 'follow']
categorical = ['race', 'sex', 'ms', 'hisp', 'educ', 'pob', 'hhid', 'reltrf', 'occ', 'majocc', 'ind', 'esr', 'urban', 'smsast', 'inddea', 'cause113', 'dayod', 'hosp', 'hospd', 'ssnyn', 'vt', 'histatus', 'hitype', 'povpct', 'stater', 'rcow', 'tenure', 'citizen', 'indalg']
smoking = ['smok100', 'agesmk', 'smokstat', 'smokhome', 'curruse', 'everuse']
misc = ['record', 'wt']

used_features = ['age', 'sex', 'stater', 'povpct', 'pob', 'race', 'urban', 'ms', 'adjinc', 'educ', 'indmort']
df = df_raw[used_features]
df.dtypes

age          int64
sex          int64
stater       int64
povpct       int64
pob          int64
race       float64
urban      float64
ms         float64
adjinc     float64
educ       float64
indmort    float64
dtype: object

In [17]:
df = df.astype({'sex':'category', 'stater': 'category', 'pob': 'category', 'race': 'category', 'urban': 'category', 'ms': 'category'})
df.dtypes

age           int64
sex        category
stater     category
povpct        int64
pob        category
race       category
urban      category
ms         category
adjinc      float64
educ        float64
indmort     float64
dtype: object

In [44]:
df_raw['indmort'].value_counts(normalize=True)

0.0    0.948718
1.0    0.051282
Name: indmort, dtype: float64

In [18]:
df_alive = df[df.indmort == 0.0]
df_dead = df[df.indmort == 1.0]
df = pd.concat([resample(df_alive, n_samples=df_dead.shape[0]*5), df_dead])
df

,age,sex,stater,povpct,pob,race,urban,ms,adjinc,educ,indmort
940866,58,1,85,2,935,1.0,1.0,1.0,2.0,10.0,0.0
116534,61,2,45,12,946,1.0,2.0,1.0,8.0,8.0,0.0
1303661,18,2,61,15,921,1.0,2.0,5.0,12.0,7.0,0.0
1434441,14,1,93,11,906,1.0,1.0,NaN,8.0,NaN,0.0
725977,15,1,22,2,934,1.0,1.0,5.0,2.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1834968,77,2,16,7,909,1.0,1.0,2.0,4.0,7.0,1.0
1834971,71,2,16,5,909,1.0,2.0,2.0,3.0,8.0,1.0
1834979,54,1,16,17,909,1.0,1.0,1.0,14.0,11.0,1.0
1835040,67,1,16,12,909,1.0,1.0,1.0,7.0,4.0,1.0


In [26]:
df.shape

(564642, 11)

In [21]:
(df.isna().sum() / df.shape[0]).sort_values()

age        0.000000
sex        0.000000
stater     0.000000
povpct     0.000000
pob        0.000000
indmort    0.000000
race       0.001617
urban      0.006333
adjinc     0.021901
educ       0.168705
ms         0.173593
dtype: float64

In [22]:
df_dropped = df.dropna(axis=0)
df_dropped = pd.get_dummies(df_dropped)

In [25]:
df_dropped.shape

(457867, 133)

In [29]:
y = df_dropped['indmort']
X = df_dropped.drop(columns=['indmort'])
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [30]:
model = GradientBoostingClassifier()
model.fit(X_train, y_train)

GradientBoostingClassifier()

In [31]:
y_pred = model.predict(X_test)

In [32]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.89      0.93      0.91     91212
         1.0       0.66      0.53      0.59     23255

    accuracy                           0.85    114467
   macro avg       0.77      0.73      0.75    114467
weighted avg       0.84      0.85      0.84    114467



In [33]:
auc(y_pred, y_test)

ValueError: x is neither increasing nor decreasing : [0. 0. 0. ... 0. 0. 0.].

In [38]:
tpr

array([nan, nan, nan])

In [39]:
import numpy as np
from sklearn import metrics
# y = np.array([1, 1, 2, 2])
# pred = np.array([0.1, 0.4, 0.35, 0.8])
metrics.roc_auc_score(y_test, y_pred)
# metrics.auc(fpr, tpr)

0.7316093066228417

In [41]:
y_test.value_counts()

0.0    91212
1.0    23255
Name: indmort, dtype: int64

In [23]:
# average_precision_score(y_test, y_pred) # sample_weight=X_test.wt